Read and plot midline data on a video

In [ ]:
import cv2
import os
import pandas as pd
import numpy as np

# 读取中线
folder_path = "Y:\\SZX\\BehaviorData\\20250716_WBI\\midline_debug\\20250716_lg9624-1gNa-002-24d-ov_002"
npz_data = np.load(folder_path + '\output-0516.npz', allow_pickle=True)
data = npz_data['arr_0'].item()
paths= {key: value['all_paths'] for key, value in data.items()}

def normalize_points(points):
    """确保骨架点是 (N,2) int32 数组"""
    arr = np.array(points, dtype=np.float32)
    arr = np.vstack(arr)                # 去掉长度为1的维度

    arr = arr.astype(np.int32)           # OpenCV 要 int32
    return arr
mid_re = [midline_points.reshape(-1,1,2)]
mid_re[0].shape
# 检查间隔（每隔n帧）
check_inv = 30
# 读取视频
cap = cv2.VideoCapture(os.path.join(folder_path, "c1.mp4"))
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx % check_inv != 0:
        frame_idx += 1
        continue
    
    if frame_idx < len(series_skeleton):
        points_raw = series_skeleton.iloc[frame_idx]  # 原始骨架
        print('456789')
        try:
            midline_points = normalize_points(points_raw)  # 返回 (N,2) int32
        except Exception as e:
            print(f"Frame {frame_idx} 骨架异常:", e)
            print(points_raw)
            frame_idx += 1
            continue

        # midline_points = np.squeeze(points)
        ; midline_points = normalize_points(points)
        cv2.polylines(frame, [midline_points.reshape(-1,1,2)], isClosed=False, color=(0, 0, 255), thickness=2)
        for (x, y) in midline_points:
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

    cv2.imshow("Skeleton Check", frame)
    
    key = cv2.waitKey(500) & 0xFF   # 播放速度 (30ms/帧)，也可以改大
    if key == ord('q'):   # 按 q 退出
        break
    elif key == ord(' '): # 按空格暂停
        cv2.waitKey(0)

    frame_idx += 1

cap.release()
cv2.destroyAllWindows()
